
# Global Parameters of the Notebook

## Paths

- Parameters related to data / model / lib paths

In [ ]:
# Lib to define paths
import os

# Define if we are running the notebook in our computer ("local")
# or in Google Colab ("remote")
RUNNING_ENV = "local"

# Base path for the rest of paths defined in the notebook
BASE_PATH = "./" if RUNNING_ENV == "local" else "/content/drive/MyDrive/Colab Notebooks/"

# Path to our lib dir
LIB_PATH = os.path.join(BASE_PATH, "lib")

# Path where we store training / test data
DATA_PATH = os.path.join(BASE_PATH, "data")

# Dir with all cached models 
# This cached models can be loaded from disk when training is skipped
MODEL_CACHE_FOLDER = os.path.join(BASE_PATH, "cached_models")

## ML parameters

- Parameters related to machine learning
- For example, batch sizes, learning rates, ...

In [ ]:
# Batch size for offline training
DATALOADER_BATCH_SIZE = 32

# Batch size for online training
ONLINE_BATCH_SIZE = 2 ** 9

# Size of random triplets sets
# It has to be a multiple of `DATALOADER_BATCH_SIZE`, so the network
# has batches of proper size
RANDOM_TRIPLETS_DATA_SIZE = DATALOADER_BATCH_SIZE * 15

# Training epochs for the random triplets model
TRAINING_EPOCHS_RANDOM = 10

# Learning rate for random triplets model
RANDOM_TRIPLET_LEARNING_RATE = 0.001
 
# Epochs for hard triplets, online training 
TRAINING_EPOCHS = 3 

# Epochs used in k-Fold Cross validation 
# k-Fold Cross validation used for parameter exploration
HYPERPARAMETER_TUNING_EPOCHS = 7

# Number of folds used in k-fold Cross Validation
NUMBER_OF_FOLDS = 4

# Margin used in the loss function
MARGIN = 0.001

# Dim of the embedding calculated by the network
EMBEDDING_DIMENSION = 2

# Learning rate para el entrenamiento con triples dificiles online
# Learning rate for hard triplets, online training
ONLINE_LEARNING_RATE = 0.01

# Number of neighbours considered in K-NN
# K-NN used for transforming embedding task to classification task 
NUMBER_NEIGHBOURS = 3

# Batch Triplet Loss Function
# This way we can choose among "hard", "all"
BATCH_TRIPLET_LOSS_FUNCTION = "hard"

# Wether or not use softplus loss function instead of vanilla triplet loss
USE_SOFTPLUS_LOSS = False

# Count all sumamnds in the mean loss or only those summands greater than zero
USE_GT_ZERO_MEAN_LOSS = True

## Section parameters

- Flags to choose if some sections will run or not
- This way we can skip some heavy computations when not needed 

In [ ]:
# Skip the training using random triplets 
SKIP_RANDOM_TRIPLETS_TRAINING = True

# Skip hyper parameter tuning for online training
SKIP_HYPERPARAMTER_TUNING = True

# Skip training and use a cached model
# Useful for testing the embedding -> classifier transformation
# Thus, when False training is not computed and a cached model
# is loaded from disk
# Cached models are stored in `MODEL_CACHE_FOLDER`
USE_CACHED_MODEL = False

## Others

In [ ]:
# Number of processes we want to use 
NUM_WORKERS = 2

# Fix random seed to make reproducible results
RANDOM_SEED = 123456789

# Auth forGoogle Drive

In [ ]:
if RUNNING_ENV == "remote":
    from google.colab import drive
    drive.mount('/content/drive')

# Importing the modules we are going to use

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.datasets as datasets

# For using pre-trained ResNets
import torchvision.models as models 
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
from pprint import pprint
import gc
import functools
import math
import seaborn as sns

# All concrete pieces we're using form sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, silhouette_score
from sklearn.model_selection import ShuffleSplit

from typing import List

# Load in the notebook all .py files that make our personal lib
# This way we keep notebook code as small as possible, and only pertinent 
# to the concrete task that this notebook solves (generic and reusable code
# goes to personal lib)
!cp -r "$LIB_PATH"/* .

# Now that files are loaded, we can import pieces of code
import core
import time
import copy
import board
import filesystem
import metrics
import loss_functions
import embedding_to_classifier
import sampler
from train_loggers import ClassificationLogger, SilentLogger, TripletLoggerOffline, TripletLoggerOnline, TrainLogger
from models import *
from visualizations import *
from tqdm.notebook import tqdm
from core import train_model_offline, train_model_online
from models import ResNet18
from loss_functions import MeanTripletBatchTripletLoss, BatchHardTripletLoss, BatchAllTripletLoss
from embedding_to_classifier import EmbeddingToClassifier
from sampler import CustomSampler

# Functions that we are going to use

In [ ]:
def show_learning_curve(training_history: dict):
    # Take two learning curves
    loss = training_history['loss']
    val_loss = training_history['val_loss']

    # Move the lists to cpu, because that's what matplotlib needs
    loss = [loss_el.cpu() for loss_el in loss]
    val_loss = [val_loss_el.cpu() for val_loss_el in val_loss]
    
    # Show graphics
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()
    
def try_to_clean_memory(): 
    torch.cuda.empty_cache() 
    gc.collect()

# Random Triplets 

- In this section, we are going to work with random triplets
- If minibatch size is $3\times n$, then:
    - Select a random element. For that element, select a random anchor and a random negative
    - Add this triple to the minibatch
    - Repeat the process $n$ times
- We use this model (determined heavily by the sampling method) as a baseline for more sophisticated models (determined heavily by more sophisticated sampling methods)

## Dataset loading

- Load training / test data
- Split training in train / validation

In [ ]:
# Transformations that we want to apply when loading the data
# Now we are only transforming images to tensors (pythorch only works with tensors)
# But we can apply here some normalizations
transform = transforms.Compose([
    transforms.ToTensor()
    # TODO -- apply some normalizations here
])

# Load the dataset
# torchvision has a method to download and load the dataset
train_dataset = torchvision.datasets.MNIST(
    root = DATA_PATH,
    train = True,
    download = True,
    transform = transform,
)

test_dataset = torchvision.datasets.MNIST(
    root = DATA_PATH,
    train = False,
    download = True,
    transform = transform,
)

# Train -> train / validation split
train_dataset, validation_dataset = core.split_train_test(train_dataset, 0.8)

# Data loaders to access the datasets
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = DATALOADER_BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True,
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size = DATALOADER_BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = DATALOADER_BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True,
)

## Exploratory Data Analysis

Show some images with their classes, to verify that data loading was properly done:

In [ ]:
imgs_to_show = 5

for _ in range(imgs_to_show):

    # Cargamos un batch de imagenes
    images, images_classes = next(iter(train_loader))

    # Nos quedamos con la primera imagen del batch
    img, img_class = images[0], images_classes[0]

    # Mostramos alguna informacion de la imagen
    print(f"Img class is: {img_class}")

    # Re-escalamos y mostramos la imagen
    img = img.reshape((28, 28))
    show_img(img, color_format_range = (-1.0, 1.0))

Show the sizes of the datasets

In [ ]:
print(f"Train dataset: {len(train_dataset)}")
print(f"Validation dataset: {len(validation_dataset)}")
print(f"Test dataset: {len(test_dataset)}")

## Random triplets generation

- For training the siamese network, we need triplets for computing *triplet loss*
- One simple way of generating triplets is randomly creating triplets (as explained before) in an offline fashion
    - Created a dataset of random triplets beforehand
    - With fixed elements 
- Later, we are going to generate triplets in an online fashion with more sophisticated methods
- In this case, we create a *pytorch* `Dataset` class, following the [official documentation]](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [ ]:
from torch.utils.data import RandomSampler

class RandomTriplets(Dataset):
    """
    Dataset en el que los elementos son triples obtenidos de forma aleatoria
    """

    def __init__(self, base_data: Dataset, custom_len: int, transform = None):
        self.base_data = base_data
        self.custom_len = custom_len
        self.transform = transform
        self.random_sampler = RandomSampler(self.base_data, replacement=True, num_samples=1, generator=None)
        
        # Por motivos de eficiencia, pre-computamos una lista de listas, de forma
        # que tengamos disponibles las listas con las posiciones de cada clase por
        # separado.
        self.posiciones_clases = self.__precompute_list_of_classes()

    def __len__(self):
        """
        Devolvemos el tamaño del dataset
        Como estamos generando triples aleatorios, devolvemos el tamaño definido
        por parametro
        """
        return self.custom_len

    def __getitem__(self, idx) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Funcion que es llamada cuando se hace dataset[idx]
        En vez de devolver una imagen (que es lo comun en esta clase dataset), 
        devolvemos un triple (anchor, positive, negative) aleatorio
        """

        # Hacemos esto por temas de eficiencia
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Tomamos una imagen aleatoria que sera el ancla
        anchor, anchor_class = self.base_data[next(iter(self.random_sampler))]

        # Tomamos una imagen de la misma clase, que sera la positiva, de forma aleatoria
        random_index = np.random.choice(self.posiciones_clases[anchor_class])
        positive, positive_class = self.base_data[random_index]

        # Tomamos una imagen de otra clase, que sera la negativa
        # Empiezo tomando una clase que no sea la del anchor
        posible_classes = list(range(10))
        posible_classes.remove(anchor_class)
        negative_class = np.random.choice(posible_classes)

        # Ahora tomamos un indice aleatorio de esta clase negativa
        random_index = np.random.choice(self.posiciones_clases[negative_class])
        negative, negative_class = self.base_data[random_index]
        
        # Generamos ahora el triple
        triplet = [anchor, positive, negative]

        # Aplicamos la transformacion dada al dataset al ejemplo que devolvemos
        if self.transform:
            triplet = [self.transform(np.array(img)) for img in triplet]

        return triplet

    def __precompute_list_of_classes(self) -> List[List[int]]:
        """
        Calcula la lista con las listas de posiciones de cada clase por separado
        """
        # Inicializamos la lista de listas
        posiciones_clases = [[] for _ in range(10)]

        # Recorremos el dataset y colocamos los indices donde corresponde
        for idx, element in enumerate(self.base_data):
            _, img_class = element
            posiciones_clases[img_class].append(idx)

        return posiciones_clases

    
class CustomReshape(object):
    """Pasamos la imagen de (28, 1, 28) a (28, 28)"""

    def __call__(self, image):
        image = image.reshape(28, 28)
        return image

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False:
   
    # Antes de modificar la base de datos para convertirla a triples
    # la guardamos, porque mas adelante nos hara falta
    old_train_dataset = train_dataset
    old_test_dataset = test_dataset

    # Necesitamos hacer reshape de las imagenes para que
    # sean (28, 28) y no (28, 1, 28)
    transform = transforms.Compose([
        transforms.ToTensor(),

        # Hacemos reshape de las imagenes para
        # que sean tensores (28, 28)
        CustomReshape(),
    ])

    # Generamos los triples aleatorios para training
    random_triplets_train = RandomTriplets(
        base_data = train_dataset,
        custom_len = RANDOM_TRIPLETS_DATA_SIZE,
        transform = transform,
    )

    train_loader = torch.utils.data.DataLoader(
        random_triplets_train,
        batch_size = DATALOADER_BACH_SIZE,
        shuffle = True,
        num_workers = NUM_WORKERS,
        pin_memory = True,
    )

    # Generamos los triples aleatorios para validacion
    random_triplets_validation = RandomTriplets(
        base_data = validation_dataset,
        custom_len = RANDOM_TRIPLETS_DATA_SIZE,
        transform = transform,
    )

    validation_loader = torch.utils.data.DataLoader(
        random_triplets_validation,
        batch_size = DATALOADER_BACH_SIZE,
        shuffle = True,
        num_workers = NUM_WORKERS,
        pin_memory = True,
    )

    # Generamos los triples aleatorios para test
    random_triplets_test = RandomTriplets(
        base_data = test_dataset,
        custom_len = RANDOM_TRIPLETS_DATA_SIZE,
        transform = transform,
    )

    test_loader = torch.utils.data.DataLoader(
        random_triplets_test,
        batch_size = DATALOADER_BACH_SIZE,
        shuffle = True,
        num_workers = NUM_WORKERS,
        pin_memory = True,
    )

    # Visualizamos algunos triples aleatorios para comprobar el funcionamiento
    custom_triplet = random_triplets_train[2]
    for i in custom_triplet :
        show_img(i, color_format_range = (-1.0, 1.0))


## Entrenamiento del modelo

In [ ]:
net = LightModel(EMBEDDING_DIMENSION)

# TODO -- fijar bien los parametros
parameters = dict()
parameters["epochs"] = TRAINING_EPOCHS_RANDOM
parameters["lr"] = RANDOM_TRIPLET_LEARNING_RATE
parameters["criterion"] = MeanTripletBatchTripletLoss(MARGIN, use_softplus = USE_SOFTPLUS_LOSS)

# Definimos el logger que queremos para el entrenamiento
logger = TripletLoggerOffline(
    net = net,
    iterations = 10 * DATALOADER_BATCH_SIZE,
    loss_func = parameters["criterion"],
)

print(net)

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False:

    training_history = train_model_offline(
        net = net,
        path = os.path.join(BASE_PATH, "tmp"),
        parameters = parameters,
        train_loader = train_loader,
        validation_loader = validation_loader,
        name = "SiameseNetwork",
        logger = logger,
        snapshot_iterations = None
    )

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False: show_learning_curve(training_history)

In [ ]:
# A partir de este punto ya no volvemos a entrenar el modelo
# Asi que lo ponemos en modo evaluacion para que no lleve 
# la cuenta de los gradientes
net.eval()

## Evaluación del modelo

- Mostramos algunas métricas fundamentales sobre el conjunto de

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False: 
    with torch.no_grad(): core.test_model(net, test_loader, parameters["criterion"])

We use the adapter to classification task to have more metrics about the model:

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False:
    with torch.no_grad():
    
        # Cargamos el dataset usando torchvision, que ya tiene el conjunto
        # preparado para descargar
        train_dataset = old_train_dataset
        test_dataset = old_test_dataset
    
        # Data loaders para acceder a los datos
        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size = DATALOADER_BACH_SIZE,
            shuffle = True,
            num_workers = NUM_WORKERS,
            pin_memory = True,
        )
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size = DATALOADER_BACH_SIZE,
            shuffle = True,
            num_workers = NUM_WORKERS,
            pin_memory = True,
        )
    
    
        classifier = EmbeddingToClassifier(
            net, 
            k = NUMBER_NEIGHBOURS, 
            data_loader = train_loader,
            embedding_dimension = EMBEDDING_DIMENSION
        )

Evaluamos este clasificador en un conjunto pequeño de imágenes de test. Más adelante tomamos métricas de dicho clasificador:

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False:
    with torch.no_grad():
    
        # Hacemos esto y no `in test_dataset[:max_iterations]`
        # para no tener que tomar todo el dataset y quedarnos con
        # una parte de el, que es un proceso mucho mas lento que usar
        # el iterador que da `in test_dataset` y parar con el contador
        counter = 0
        max_iterations = 20
    
        for img, img_class in test_dataset:
            predicted_class = classifier.predict(img)
            print(f"Etiqueta verdadera: {img_class}, etiqueta predicha: {predicted_class[0]}")
    
            # Actualizamos el contador
            counter += 1
            if counter == max_iterations: break

## Plot del embedding

- Aprovechamos el cálculo realizado en la clase que realiza la adaptación a clasificación para mostrar gráficamente el embedding calculado
- Esta gráfica solo la visualizamos cuando el embedding tiene dimensión 2

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False: 
    with torch.no_grad(): classifier.scatter_plot()

## Evaluación del clasificador obtenido

- Ahora que hemos adaptado el modelo para usarlo como clasificador, podemos consultar ciertas métricas de clasificación

In [ ]:
from sklearn.metrics import confusion_matrix

def calculate_roc_auc(true_labels_prob: np.array, predicted_labels_prob: np.array) -> float:
    """
    Calcula el area bajo la curva ROC, dadas las etiquetas verdaderas y las
    etiqeutas predichas por un modelo
    
    Las listas de etiquetas deben ser etiquetas probabilisticas
    """
    return roc_auc_score(true_labels_prob, predicted_labels_prob, multi_class = "ovo")
    
def calculate_accuracy(true_labels: np.array, predicted_labels: np.array) -> float:
    """
    Calcula el accuracy, dadas las etiquetas verdaderas y las
    etiqeutas predichas por un modelo
    """
    return accuracy_score(true_labels, predicted_labels)

def calculate_silhouette(x, y):
    """Calcula el indice de silhouette para el embedding calculado por el modelo"""
    return silhouette_score(x, y)

def evaluate_model(model, train_loader, test_loader) -> dict:
    """
    Evalua, usando distintas metricas, el modelo que hemos entrenado
    Tambien evaluamos el embedding obtenido, no solo el clasificador
    
    Devuelve un diccionario con todas las metricas calculadas con el modelo
    """
    
    # Diccionario en el que vamos a almacenar todas las metricas
    metrics = dict()
    
    # Tomamos los arrays en formato adecuado para calcular ciertas metricas
    x_train, y_train = EmbeddingToClassifier(net, k = NUMBER_NEIGHBOURS, data_loader = train_loader, embedding_dimension = EMBEDDING_DIMENSION).prepare_data_for_sklearn()
    x_test, y_test = EmbeddingToClassifier(net, k = NUMBER_NEIGHBOURS, data_loader = test_loader, embedding_dimension = EMBEDDING_DIMENSION).prepare_data_for_sklearn()
    
    # Empezamos usando el modelo para realizar las predicciones+
    # Usamos predicciones probabilisticas pues estas son fundamentales para
    # la metrica roc auc
    train_predicted_labels_prob = model.predict_proba_using_embedding(x_train)
    test_predicted_labels_prob = model.predict_proba_using_embedding(x_test)

    # Tomamos ahora las etiqeutas sin probabilidad
    train_predicted_labels= model.predict_using_embedding(x_train)
    test_predicted_labels = model.predict_using_embedding(x_test)
    
    # Tomamos las metricas de accuracy
    metrics["train_acc"] = calculate_accuracy(y_train, train_predicted_labels)
    metrics["test_acc"] = calculate_accuracy(y_test, test_predicted_labels)
    
    # Tomamos las areas bajo la curva ROC
    metrics["train_roc_auc"] = calculate_roc_auc(y_train, train_predicted_labels_prob)
    metrics["test_roc_auc"] = calculate_roc_auc(y_test, test_predicted_labels_prob)
    
    # Tomamos el indice de silhouette
    metrics["train_silhouette"] = calculate_silhouette(x_train, y_train)
    metrics["test_silhouette"] = calculate_silhouette(x_test, y_test)
    
    #Calculamos Matriz de confusion
    metrics["train_confusion_matrix"]=confusion_matrix(y_train, train_predicted_labels)
    metrics["test_confusion_matrix"]=confusion_matrix(y_test, test_predicted_labels)
    
    # Devolvemos las metricas en formato diccionario, que nos va a ser comodo para
    # pasarlas a tablas y para mostrar muchas metricas simultaneamente
    return metrics


def plot_confusion_matrix(train_confusion_matrix,test_confusion_matrix):
    """
    Mostramos graficamente (mapa de calor) la matriz de confusion
    
    Podriamos mostrar directamente los valores de la matriz, pero al estar
    trabajando con 25 clases es mas dificil de interpretar
    """
    
    # Mostramos graficamente las dos matrices de confusion
    color_palette = sns.color_palette("viridis", as_cmap=True)
    print("Matriz de confusion en TRAIN")
    sns.heatmap(train_confusion_matrix, annot = False, cmap = color_palette, fmt="d")
    plt.show()
    
    print("Matriz de confusion en TEST")
    sns.heatmap(test_confusion_matrix, annot = False, cmap = color_palette)
    plt.show()

In [ ]:
# Controlamos si queremos ejecutar esta seccion o no 
if SKIP_RANDOM_TRIPLETS_TRAINING is False:
    with torch.no_grad():
    
        classifier.embedder.set_permute(False)
    
        metrics = evaluate_model(classifier, train_loader, test_loader)
        pprint(metrics)
    
        classifier.embedder.set_permute(True)

# Online Batch Triplets

## Choose the loss function to use 

- We have so many combinations for loss functions that is not feasible to use one Colab section for each
- Combinations depend on:
    1. Batch hard vs Batch all 
    2. Classical triplet loss vs Softplus loss
    3. All summands mean vs Only > 0 summands mean

In [ ]:
batch_loss_function = None
if BATCH_TRIPLET_LOSS_FUNCTION == "hard":
    batch_loss_function = BatchHardTripletLoss(MARGIN, use_softplus = USE_SOFTPLUS_LOSS, use_gt_than_zero_mean = USE_GT_ZERO_MEAN_LOSS) 
if BATCH_TRIPLET_LOSS_FUNCTION == "all":
    batch_loss_function = BatchAllTripletLoss(MARGIN, use_softplus = USE_SOFTPLUS_LOSS, use_gt_than_zero_mean =  USE_GT_ZERO_MEAN_LOSS) 

# Sanity check
if batch_loss_function is None:
    raise Exception(f"BATCH_TRIPLET_LOSS global parameter got unexpected value: {BATCH_TRIPLET_LOSS_FUNCTION}")

## Load again the data

- With this online loss function we no longer need to compute triplets in an offline way
- So we load the data again, so we have the original data without precomputed triplets 
- Also, use custom sampler to choose, for each minibatch, P classes and K elements for each class

In [ ]:
# Transformaciones que queremos aplicar al cargar los datos
# Ahora solo pasamos las imagenes a tensores, pero podriamos hacer aqui normalizaciones
transform = transforms.Compose([
    transforms.ToTensor(),
    # TODO -- aqui podemos añadir la normaliazcion de datos
])

# Cargamos el dataset usando torchvision, que ya tiene el conjunto
# preparado para descargar
train_dataset = torchvision.datasets.MNIST(
    root = DATA_PATH,
    train = True,
    download = True,
    transform = transform,
)

test_dataset = torchvision.datasets.MNIST(
    root = DATA_PATH,
    train = False,
    download = True,
    transform = transform,
)

# Separamos train en train y validacion
train_dataset, validation_dataset = core.split_train_test(train_dataset, 0.8)

# Dont work with subsets
train_dataset, validation_dataset = train_dataset.dataset, validation_dataset.dataset

# Data loaders para acceder a los datos
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = ONLINE_BATCH_SIZE,
    #shuffle = True,
    num_workers = NUM_WORKERS, 
    pin_memory = True,
    # TODO -- using magic numbers
    sampler = CustomSampler(3, 16, train_dataset)
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size = ONLINE_BATCH_SIZE,
    #shuffle = True,
    num_workers = NUM_WORKERS, 
    pin_memory = True,
    # TODO -- using magic numbers
    # Maybe here I can use default sampler?
    sampler = CustomSampler(3, 16, validation_dataset)
)

test_loader = torch.utils.data.DataLoader(
  test_dataset,
  batch_size = ONLINE_BATCH_SIZE,
  shuffle = True,
  num_workers = NUM_WORKERS,
  pin_memory = True,
)

In [ ]:
# TODO -- remove this check
for element in train_loader:
    print(f"Element loaded: {element}")


## Hyperparameter tuning

In [ ]:
def custom_cross_validation(net, parameters, train_dataset, k):
    """Funcion propia para hacer k-fold cross validation de una red convolucional"""

    # Definimos la forma en la que vamos a hacer el split de los folds
    ss = ShuffleSplit(n_splits=k, test_size=0.25, random_state=RANDOM_SEED)
    
    # Lista en la que guardamos las perdidas encontradas en cada fold
    losses = []

    # Iteramos usando el split que nos da sklearn
    for train_index, validation_index in ss.split(train_dataset):
        
        # Tenemos los indices de los elementos, asi que tomamos los dos datasets
        # usando dichos indices
        train_fold = [train_dataset[idx] for idx in train_index]
        validation_fold = [train_dataset[idx] for idx in validation_index]

        # Transformamos los datasets en dataloaders
        train_loader = torch.utils.data.DataLoader(
            train_fold,
            batch_size = ONLINE_BATCH_SIZE,
            shuffle = True,
            num_workers = NUM_WORKERS,
            pin_memory = True,
        )
        validation_loader = torch.utils.data.DataLoader(
            validation_fold,
            batch_size = ONLINE_BATCH_SIZE,
            shuffle = True,
            num_workers = NUM_WORKERS,
            pin_memory = True,
        )

        # Entrenamos la red
        _ = train_model_online(
            net = net,
            path = os.path.join(BASE_PATH, "tmp"),
            parameters = parameters,
            train_loader = train_loader,
            validation_loader = validation_loader,
            name = "SiameseNetworkOnline",
            logger = SilentLogger(),
            snapshot_iterations = None
        )

        # Evaluamos la red en el fold de validacion
        net.eval()
        loss = metrics.calculate_mean_triplet_loss_online(net, validation_loader, parameters["criterion"], 1.0)
        loss = float(loss) # Pasamos el tensor de un unico elemento a un float simple

        # Añadimos el loss a nuestra lista
        losses.append(loss)
    
    # Devolvemos el array en formato numpy para que sea mas comodo trabajar con ella
    return np.array(losses)


In [ ]:
# Controlamos si queremos realizar el hyperparameater tuning o no
if SKIP_HYPERPARAMTER_TUNING is False:

    # Para controlar que parametros ya hemos explorado y queremos saltar
    already_explored_parameters = [
        # Embedding dimension, learning rate, margin
        (2, 0.0001, 0.01),
        (2, 0.0001, 1),
        (3, 0.0001),
    ]

    # Parametros que queremos mover
    #margin_values = [0.01, 0.1, 1.0]
    # TODO -- volver a poner todos los valores
    margin_values = [1.0]
    learning_rate_values = [0.0001, 0.001, 0.01]
    embedding_dimension_values = [2, 3, 4]
    
    # Parametros que fijamos de antemano
    epochs = HYPERPARAMETER_TUNING_EPOCHS
    
    # Llevamos la cuenta de los mejores parametros y el mejor error encontrados hasta
    # el momento
    best_loss = None
    best_parameters = {
        "embedding_dimension": None,
        "lr": None,
        "margin": None
    }
    
    # Exploramos las combinaciones de parametros
    for margin in margin_values:
        for learning_rate in learning_rate_values:
            for embedding_dimension in embedding_dimension_values:
        
                print(f"Optimizando para margin: {margin}, lr: {learning_rate}, embedding_dim: {embedding_dimension}")

                # Comprobamos si tenemos que saltarnos el calculo de algun valor
                # porque ya se haya hecho
                if (embedding_dimension, learning_rate, margin) in already_explored_parameters:
                    print("\tSaltando este calculo porque ya se realizo")
                    continue
                
                # Definimos el modelo que queremos optimizar
                net = ResNet18(embedding_dimension)
                
                # En este caso, al no estar trabajando con los minibatches
                # (los usamos directamente como nos los da pytorch), no tenemos
                # que manipular los tensores
                net.set_permute(False)
                
                parameters = dict()
                parameters["epochs"] = epochs
                parameters["lr"] = learning_rate
                parameters["criterion"] = BatchHardTripletLoss(margin)
                logger = SilentLogger()
    
                # Usamos nuestra propia funcion de cross validation para validar el modelo
                losses = custom_cross_validation(net, parameters, train_dataset, k = NUMBER_OF_FOLDS) 
                print(f"El loss conseguido es {losses.mean()}")
                print("")
            
                # Comprobamos si hemos mejorado la funcion de perdida
                # En cuyo caso, actualizamos nuestra estructura de datos y, sobre todo, mostramos
                # por pantalla los nuevos mejores parametros
                basic_condition = math.isnan(losses.mean()) is False             # Si es NaN no entramos al if
                enter_condition = best_loss is None or losses.mean() < best_loss # Entramos al if si mejoramos la perdida
                compound_condition = basic_condition and enter_condition
                if compound_condition:
                
                    # Actualizamos nuestra estructura de datos
                    best_loss = losses.mean()
                    best_parameters = {
                        "embedding_dimension": embedding_dimension,
                        "lr": learning_rate,
                        "margin": margin,
                    }
            
                    # Mostramos el cambio encontrado
                    print("==> ENCONTRADOS NUEVOS MEJORES PARAMETROS")
                    print(f"Mejores parametros: {best_parameters}")
                    print(f"Mejor loss: {best_loss}")
            
                


## Entrenamiento online

In [ ]:
net = LightModel(EMBEDDING_DIMENSION)

# En este caso, al no estar trabajando con los minibatches
# (los usamos directamente como nos los da pytorch), no tenemos
# que manipular los tensores
net.set_permute(False)

# Parameters of the training
parameters = dict()
parameters["epochs"] = TRAINING_EPOCHS
parameters["lr"] = ONLINE_LEARNING_RATE

# We use the loss function that depends on the global parameter BATCH_TRIPLET_LOSS_FUNCTION
parameters["criterion"] = batch_loss_function

# Definimos el logger que queremos para el entrenamiento
logger = TripletLoggerOnline(
    net = net,
    iterations = -1,
    loss_func = parameters["criterion"],
    train_percentage = 0.01,
    validation_percentage = 1.0,
)

print(net)

In [ ]:
import torch

# Entrenamos solo si lo especifica
# el parametro que controla el uso de cache
if USE_CACHED_MODEL is False:

    # Para saber cuanto tarda
    ts = time.time()

    torch.jit.script(
    training_history = train_model_online(
        net = net,
        path = os.path.join(BASE_PATH, "tmp"),
        parameters = parameters,
        train_loader = train_loader,
        validation_loader = validation_loader,
        name = "SiameseNetworkOnline",
        logger = logger,
        snapshot_iterations = None
    ))

    # Calculamos cuanto ha tardado
    te = time.time()
    print(f"Ha tardado {te - ts}")
    
    # Actualizamos la cache del modelo
    filesystem.save_model(net, MODEL_CACHE_FOLDER, "online_model_cached")
    
# Nos saltamos el entrenamiento y cargamos el modelo desde la cache
else:
       
    net = filesystem.load_model(
        os.path.join(MODEL_CACHE_FOLDER, "online_model_cached"), 
        lambda: ResNet18(EMBEDDING_DIMENSION)
    )
    
    # Tenemos que cargar la red en la memoria correspondiente
    device = core.get_device()
    net.to(device)
    
# A partir de este punto no hacemos entrenamiento
# asi que ponemos la red en modo evaluacion para que 
# no vaya almacenando los gradientes
net.eval()

In [ ]:
# Entrenamos solo si lo especifica
# el parametro que controla el uso de cache
if USE_CACHED_MODEL is False:
        
    # Al principio tenemos un pico de error que hace 
    # que la grafica no sea legible. Ignoramos las primeras metricas
    # de error para que la grafica sea legible
    cut = 5
    training_history["loss"] = training_history["loss"][cut:]
    training_history["val_loss"] = training_history["val_loss"][cut:]

    show_learning_curve(training_history)

## Evaluación del modelo

- Mostramos algunas métricas fundamentales sobre el conjunto de

In [ ]:
with torch.no_grad():
    net.set_permute(False)
    
    core.test_model_online(net, test_loader, parameters["criterion"], online = True)
    
    net.set_permute(True)

In [ ]:
with torch.no_grad():
    try_to_clean_memory()
    classifier = EmbeddingToClassifier(net, k = NUMBER_NEIGHBOURS, data_loader = train_loader, embedding_dimension = EMBEDDING_DIMENSION)

Evaluamos este clasificador en un conjunto pequeño de imágenes de test. Más adelante tomamos métricas de dicho clasificador:

In [ ]:
with torch.no_grad():
    # Hacemos esto y no `in test_dataset[:max_iterations]`
    # para no tener que tomar todo el dataset y quedarnos con
    # una parte de el, que es un proceso mucho mas lento que usar
    # el iterador que da `in test_dataset` y parar con el contador
    counter = 0
    max_iterations = 20
    
    for img, img_class in test_dataset:
        predicted_class = classifier.predict(img)
        print(f"Etiqueta verdadera: {img_class}, etiqueta predicha: {predicted_class[0]}")
    
        # Actualizamos el contador
        counter += 1
        if counter == max_iterations: break

## Plot del embedding

- Aprovechamos el cálculo realizado en la clase que realiza la adaptación a clasificación para mostrar gráficamente el embedding calculado
- Esta gráfica solo la visualizamos cuando el embedding tiene dimensión 2

In [ ]:
with torch.no_grad():
    classifier.scatter_plot()

## Evaluación del clasificador obtenido

- Ahora que hemos adaptado el modelo para usarlo como clasificador, podemos consultar ciertas métricas de clasificación

In [ ]:
with torch.no_grad():
    try_to_clean_memory()
    classifier.embedder.set_permute(False)
    
    metrics = evaluate_model(classifier, train_loader, test_loader)
    pprint(metrics)
    
    classifier.embedder.set_permute(True)